# Data Analyst Associate Practical Exam Submission - Food Claims Process

## Introduction

Vivendo is a fast food chain in Brazil with over 200 outlets.
Customers often claim compensation from the company for food poisoning.
The legal team processes these claims. The legal team has offices in four locations.
The legal team wants to improve how long it takes to reply to customers and close claims.
The legal team processes these claims. The legal team has offices in four locations

### Questions
-  How long it takes to reply to customers and close claims?
-  How does the number of claims differ across locations?
-  How each location differs in the time it takes to close claims?

## Data
The dataset contains one row for each claim. The dataset table consist of 8 columns;
- **claim_id**: Character. Unique identifier of the claim.
- **time_to_close**: Numeric. The number of days to close the claim.
- **claim_amount**: Numeric. Initial amount in the currency of Brazil.
- **amount_paid**: Numeric. Final amount in the same currency.
- **location**: Character. Location of the claim, one of "RECIFE", "SAO LUIS", "FORTALEZA", or "NATAL".
- **individuals_on_claim**: Numeric. Number of individuals on this claim.
- **linked_cases**: Binary. 1 for TRUE and 0 for FALSE. Whether this claim is linked to other cases.
- **cause**: Character. Cause of the food poisoning. One of "vegetable", "meat" or "unknown".

## Data Overview
First, we will look at the rows and columns on the table and then inspect the dataset to detect if there are any issues before working with the data.

In [147]:
SELECT *
FROM 'food_claims.csv';

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
0,0001,317,R$ 74474.55,51231.37,RECIFE,15,FALSE,unknown
1,0002,195,R$ 52137.83,42111.3,FORTALEZA,12,TRUE,unknown
2,0003,183,R$ 24447.2,23986.3,SAO LUIS,10,TRUE,meat
3,0004,186,R$ 29006.28,27942.72,FORTALEZA,11,FALSE,meat
4,0005,138,R$ 19520.6,16251.06,RECIFE,11,FALSE,vegetable
...,...,...,...,...,...,...,...,...
1995,1996,176,R$ 28982.3,24265.02,RECIFE,10,FALSE,meat
1996,1997,166,R$ 5188.44,4772.77,FORTALEZA,2,TRUE,meat
1997,1998,179,R$ 11975.85,10087.81,RECIFE,4,TRUE,meat
1998,1999,162,R$ 23516.28,23310.24,RECIFE,9,FALSE,meat


Checking the claim_id column.

In [148]:
SELECT COUNT(claim_id) AS num_of_ids,
    (SELECT COUNT(*)
     FROM claims
     WHERE claim_id = 'NA') AS num_of_null_ids
FROM claims
WHERE claim_id != 'NA';

,num_of_ids,num_of_null_ids
0,2000,0


Found 2000 claims. claim_id rows consist of integers and not found any null values.

Checking time_to_close column.

In [149]:
SELECT COUNT(time_to_close) AS num_of_TTC,
    (SELECT COUNT(*)
     FROM claims
     WHERE time_to_close = 'NA') AS num_of_null_TTC
FROM claims
WHERE time_to_close IS NOT NULL;

,num_of_TTC,num_of_null_TTC
0,2000,0


Checking claim_amount column.

In [150]:
SELECT COUNT(claim_amount) AS num_of_CA,
    (SELECT COUNT(*)
     FROM claims
     WHERE claim_amount = 'NA') AS num_of_null_CA
FROM claims
WHERE claim_amount != 'NA';

,num_of_CA,num_of_null_CA
0,2000,0


Checking amount_paid column.

In [151]:
SELECT COUNT(amount_paid) AS num_of_AP,
    (SELECT COUNT(amount_paid)
     FROM 'food_claims.csv'
     WHERE amount_paid = 'NA') AS num_of_null_AP
FROM 'food_claims.csv'
WHERE amount_paid != 'NA';

,num_of_AP,num_of_null_AP
0,1964,36


Found 36 rows of column amount_paid are null.

Checking location column.

In [152]:
SELECT COUNT(location) AS num_of_Loc,
    (SELECT COUNT(*)
     FROM claims
     WHERE location IS NULL) AS num_of_null_Loc
FROM claims;

,num_of_Loc,num_of_null_Loc
0,2000,0


Checking individuals_on_claim column.

In [153]:
SELECT COUNT(individuals_on_claim) AS num_of_indiv,
    (SELECT COUNT(*)
     FROM claims
     WHERE individuals_on_claim IS NULL) AS num_of_null_indiv
FROM claims;

,num_of_indiv,num_of_null_indiv
0,2000,0


Checking linked_cases column.

In [154]:
SELECT COUNT(linked_cases) AS num_of_linked,
    (SELECT COUNT(*)
     FROM claims
     WHERE linked_cases = 'NA') AS num_of_null_linked
FROM claims
WHERE linked_cases != 'NA';

,num_of_linked,num_of_null_linked
0,1974,26


Found 26 rows of column linked_cases are null.

Checking cause column.

In [155]:
SELECT COUNT(cause) AS num_of_cause,
    (SELECT COUNT(*)
     FROM claims
     WHERE cause != 'unknown' AND cause != 'vegetable' AND cause != 'meat') AS num_of_null_cause
FROM claims
WHERE cause = 'unknown' OR cause = 'vegetable' OR cause = 'meat';

,num_of_cause,num_of_null_cause
0,1970,30


We found 30 null rows of cause column.

In [156]:
SELECT cause
FROM claims
WHERE cause != 'unknown' AND cause != 'vegetable' AND cause != 'meat';

,cause
0,Meat
1,VEGETABLES
2,Meat
3,VEGETABLES
4,Meat
5,Meat
6,VEGETABLES
7,Meat
8,VEGETABLES
9,Meat


As we can see from the table, values are not null. We need to transform the values to make them the same as the other ones. 

## Data Cleaning

amount_paid, linked_cases and cause columns are including null values. Before working on the dataset we need to prepare the dataset.

We are starting with amount_paid column. Here, we need to fill the NA values with the median of amount_paid.

In [157]:
WITH clear_amount_paid AS (
	SELECT *
	FROM claims
	WHERE amount_paid != 'NA'
)
	
SELECT PERCENTILE_CONT(0.5) 
	WITHIN GROUP (ORDER BY CAST(amount_paid AS NUMERIC)) AS median
FROM clear_amount_paid;

,median
0,20105.7


Median value of amount_paid is calculated.

In [158]:
WITH cte_median AS (
  SELECT *,
	CASE
		WHEN amount_paid = 'NA' THEN (SELECT median FROM median_amount_paid)
		ELSE amount_paid
	END AS new_amount_paid
  FROM claims
)

SELECT claim_id, time_to_close, claim_amount, new_amount_paid AS amount_paid, location, individuals_on_claim, linked_cases, cause
FROM cte_median;

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
0,0001,317,R$ 74474.55,51231.37,RECIFE,15,FALSE,unknown
1,0002,195,R$ 52137.83,42111.3,FORTALEZA,12,TRUE,unknown
2,0003,183,R$ 24447.2,23986.3,SAO LUIS,10,TRUE,meat
3,0004,186,R$ 29006.28,27942.72,FORTALEZA,11,FALSE,meat
4,0005,138,R$ 19520.6,16251.06,RECIFE,11,FALSE,vegetable
...,...,...,...,...,...,...,...,...
1995,1996,176,R$ 28982.3,24265.02,RECIFE,10,FALSE,meat
1996,1997,166,R$ 5188.44,4772.77,FORTALEZA,2,TRUE,meat
1997,1998,179,R$ 11975.85,10087.81,RECIFE,4,TRUE,meat
1998,1999,162,R$ 23516.28,23310.24,RECIFE,9,FALSE,meat


In [159]:
SELECT *
FROM claims;

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
0,0001,317,R$ 74474.55,51231.37,RECIFE,15,FALSE,unknown
1,0002,195,R$ 52137.83,42111.3,FORTALEZA,12,TRUE,unknown
2,0003,183,R$ 24447.2,23986.3,SAO LUIS,10,TRUE,meat
3,0004,186,R$ 29006.28,27942.72,FORTALEZA,11,FALSE,meat
4,0005,138,R$ 19520.6,16251.06,RECIFE,11,FALSE,vegetable
...,...,...,...,...,...,...,...,...
1995,1996,176,R$ 28982.3,24265.02,RECIFE,10,FALSE,meat
1996,1997,166,R$ 5188.44,4772.77,FORTALEZA,2,TRUE,meat
1997,1998,179,R$ 11975.85,10087.81,RECIFE,4,TRUE,meat
1998,1999,162,R$ 23516.28,23310.24,RECIFE,9,FALSE,meat


amount_paid column is now clear. Now, we will fill the null linked_cases values with FALSE.

In [160]:
WITH new_linked_cases AS (
  SELECT *,
	CASE
		WHEN linked_cases = 'NA' THEN 'FALSE'::TEXT
		ELSE linked_cases
	END AS new_linked_cases
  FROM claims
)

SELECT claim_id, time_to_close, claim_amount, amount_paid, location, individuals_on_claim, new_linked_cases AS linked_cases, cause
FROM new_linked_cases;

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
0,0001,317,R$ 74474.55,51231.37,RECIFE,15,FALSE,unknown
1,0002,195,R$ 52137.83,42111.3,FORTALEZA,12,TRUE,unknown
2,0003,183,R$ 24447.2,23986.3,SAO LUIS,10,TRUE,meat
3,0004,186,R$ 29006.28,27942.72,FORTALEZA,11,FALSE,meat
4,0005,138,R$ 19520.6,16251.06,RECIFE,11,FALSE,vegetable
...,...,...,...,...,...,...,...,...
1995,1996,176,R$ 28982.3,24265.02,RECIFE,10,FALSE,meat
1996,1997,166,R$ 5188.44,4772.77,FORTALEZA,2,TRUE,meat
1997,1998,179,R$ 11975.85,10087.81,RECIFE,4,TRUE,meat
1998,1999,162,R$ 23516.28,23310.24,RECIFE,9,FALSE,meat


linked_cases column is now clear.

In [161]:
WITH new_cause AS (
SELECT *,
	CASE
		WHEN LOWER(cause) = 'vegetable' THEN 'vegetables'
		ELSE LOWER(TRIM(cause))
	END AS new_cause
	FROM claims
)

SELECT claim_id, time_to_close, claim_amount, amount_paid, location, individuals_on_claim, linked_cases, new_cause AS cause
FROM new_cause;

,claim_id,time_to_close,claim_amount,amount_paid,location,individuals_on_claim,linked_cases,cause
0,0001,317,R$ 74474.55,51231.37,RECIFE,15,FALSE,unknown
1,0002,195,R$ 52137.83,42111.3,FORTALEZA,12,TRUE,unknown
2,0003,183,R$ 24447.2,23986.3,SAO LUIS,10,TRUE,meat
3,0004,186,R$ 29006.28,27942.72,FORTALEZA,11,FALSE,meat
4,0005,138,R$ 19520.6,16251.06,RECIFE,11,FALSE,vegetables
...,...,...,...,...,...,...,...,...
1995,1996,176,R$ 28982.3,24265.02,RECIFE,10,FALSE,meat
1996,1997,166,R$ 5188.44,4772.77,FORTALEZA,2,TRUE,meat
1997,1998,179,R$ 11975.85,10087.81,RECIFE,4,TRUE,meat
1998,1999,162,R$ 23516.28,23310.24,RECIFE,9,FALSE,meat


Data cleaning process is completed successfully. Null values are filled and varying text values are in a standard. Now, we can start to the analysis.

## Exploratary Analysis

In this section, we will calculate the summary functions of the dataset and get insights.

To answer this question we need to use the time_to_close column of the table. In this column, every value of rows indicates the number of days to close the claim.

## - How long it takes to reply to customers and close claims?

In [162]:
SELECT 
    CASE
        WHEN time_to_close < 90 THEN 'Within 3 Months'
        WHEN time_to_close < 180 THEN 'Within 6 Months'
        WHEN time_to_close < 270 THEN 'Within 9 Months'
        WHEN time_to_close < 365 THEN 'Within 1 Year'
        WHEN time_to_close < 730 THEN 'Within 2 Year'
    END AS time_to_close_formatted, 
    COUNT(*) as count_of_claims
FROM claims
GROUP BY time_to_close_formatted
ORDER BY count_of_claims DESC;


,time_to_close_formatted,count_of_claims
0,Within 6 Months,1022
1,Within 9 Months,850
2,Within 1 Year,110
3,Within 2 Year,11
4,Within 3 Months,7


We formatted the time_to_close column and get the count of claims according to the format. Now, the table is ready to be interpreted with a plot.

,time_to_close_formatted,count_of_claims
0,Within 6 Months,1022
1,Within 9 Months,850
2,Within 1 Year,110
3,Within 2 Year,11
4,Within 3 Months,7


This bar plot indicates the number of claims and the time to close the claims. X-axis refers to the number of claims and the y-axis refers to the time to close the claims.

### Insights

- The bar plot shows us most of the claims can be closed in 9 months. But there are also  100+ cases that can be up to 1-2 years to close them. 

## - How does the number of claims differ across locations?

In [164]:
SELECT location, COUNT(*) AS count
FROM claims
GROUP BY location
ORDER BY count DESC;


,location,count
0,RECIFE,885
1,SAO LUIS,517
2,FORTALEZA,311
3,NATAL,287


This table shows us the which legal branch is have how many claims.

,location,count
0,RECIFE,885
1,SAO LUIS,517
2,FORTALEZA,311
3,NATAL,287


In this bar plot, we can see how the number of claims is distributed across the locations. the x-axis refers to the count of claims and the y-axis refers to the locations.

### Insights

- This bar plot shows us most of the claims given to the RECIFE branch with 885 claims.
- SAO LUIS branch comes after the RECIFE with 517 claims
- FORTALEZA AND NATAL branches have the rest of the claims almost equally.

## -  How each location differs in the time it takes to close claims?

In [166]:
SELECT location, ROUND(AVG(time_to_close)) AS avg_time_to_close
FROM claims
GROUP BY location
ORDER BY AVG(time_to_close) DESC;

,location,avg_time_to_close
0,SAO LUIS,187.0
1,NATAL,186.0
2,FORTALEZA,185.0
3,RECIFE,185.0


,location,avg_time_to_close
0,SAO LUIS,187.0
1,NATAL,186.0
2,FORTALEZA,185.0
3,RECIFE,185.0


This bar plot indicates how the average time to close differs across the locations. X-axis refers to the average time to close and the y-axis refers to the locations.

### Insights

- According to the bar plot, all the branches have almost the same average time to close claims.

## - What is the distribution of claim causes

In [168]:
SELECT cause, COUNT(*) AS count
FROM claims
GROUP BY cause
ORDER BY count DESC;

,cause,count
0,meat,957
1,unknown,713
2,vegetables,330


This table shows us the distribution of the causes.

,cause,count
0,meat,957
1,unknown,713
2,vegetables,330


This bar plot indicates how the causes distributed. X-axis refers to the count of the cause and y-axis refers to the cause.

### Insights

- According to the plot, the most common cause is  meat with 957 causes.
- After the meat, unknown causes come with 713 causes.
- Vegetables are the third common cause with 330 causes.


## Conclusion

After the exploratory data analysis process, we found the most common cause is meat. Most of the claims can be solved in 9 months. RECIFE is the branch that had the most claims. And the average time to claims according to the branches are almost the same.